# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [39]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

chrome_option = webdriver.chrome.options.Options()
# chrome_option.add_argument('--headless')
browser = webdriver.Chrome(executable_path='chromedriver', chrome_options=chrome_option)
url = 'https://erdb.epa.gov.tw/DataRepository/EnvMonitor/AirQualityMonitorMonData.aspx'
browser.get(url)

D:\Users\Linfe\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [41]:
rblCondition = browser.find_element_by_id('ctl00_ContentPlaceHolder1_ucSearchCondition_rblCondition_1')
rblCondition.click()
time.sleep(0.5)
AirArea = Select(browser.find_element_by_id('ctl00_ContentPlaceHolder1_ucSearchCondition_ddlAirArea'))
AirArea.select_by_value('2')
time.sleep(0.5)
ddCounty = Select(browser.find_element_by_id('ctl00_ContentPlaceHolder1_ucSearchCondition_ddlCounty'))
ddCounty.select_by_value('63000')
time.sleep(0.5)
ddTownShip = Select(browser.find_element_by_id('ctl00_ContentPlaceHolder1_ucSearchCondition_ddlTownship'))
ddTownShip.select_by_value('63000110')
time.sleep(0.5)
ddlSite = Select(browser.find_element_by_id('ctl00_ContentPlaceHolder1_ucSearchCondition_ddlSite'))
ddlSite.select_by_value('全部')
time.sleep(0.5)
ddlYear = Select(browser.find_element_by_id('ctl00_ContentPlaceHolder1_ucSearchCondition_ddlYear'))
ddlYear.select_by_value('2018')
time.sleep(0.5)
ddlMonth = Select(browser.find_element_by_id('ctl00_ContentPlaceHolder1_ucSearchCondition_ddlMonth'))
ddlMonth.select_by_value('01')
time.sleep(0.5)
ddlYearE = Select(browser.find_element_by_id('ctl00_ContentPlaceHolder1_ucSearchCondition_ddlYearE'))
ddlYearE.select_by_value('2018')
time.sleep(0.5)
ddlMonthE = Select(browser.find_element_by_id('ctl00_ContentPlaceHolder1_ucSearchCondition_ddlMonthE'))
ddlMonthE.select_by_value('08')
time.sleep(0.5)
browser.find_element_by_id('ctl00_ContentPlaceHolder1_imgSearch').click()

In [112]:
html_source = browser.page_source
soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_ = 'gvColspan')
wsnw = table.find_all('tr')
Res, SO2s, COs = {}, {}, {}
for k, v in enumerate(wsnw):
    if k == 0: continue
    town = v.find(style='width:10%;').text
    if not town == '士林': continue
    # print(v)
    date = v.find(style='width: 13%; display: none;').text
    IsNA = v.find_all(class_='IsNA')
    SO2 = IsNA[0].text
    CO = IsNA[1].text
    SO2s[date] = SO2
    COs[date] = CO
Res['SO2'] = SO2s
Res['CO'] = COs
for k, v in Res.items():
    print(k, v)

browser.quit()

SO2 {'2018/08': '2.4', '2018/07': '2.2', '2018/06': '2.7', '2018/05': '3.1', '2018/04': '2.3', '2018/03': '2.2', '2018/02': '1.9'}
CO {'2018/08': '0.30', '2018/07': '0.21', '2018/06': '0.29', '2018/05': '0.34', '2018/04': '0.38', '2018/03': '0.40', '2018/02': '0.44'}


In [113]:
import pandas as pd
pd.DataFrame(Res)

,SO2,CO
2018/02,1.9,0.44
2018/03,2.2,0.40
2018/04,2.3,0.38
2018/05,3.1,0.34
2018/06,2.7,0.29
2018/07,2.2,0.21
2018/08,2.4,0.30
